## Installation

1. If you haven't already installed Python and Jupyter Notebook:   
    1. Get Python3 from [Python.org](https://www.python.org/downloads/). **Tensorflow does not yet work with Python 3.7, so you _must_ get Python 3.6.** See https://github.com/tensorflow/tensorflow/issues/20517 for updates on 3.7 support.
    1. In Terminal, run `python3 -m pip install jupyter`
    1. In Terminal, cd to the folder in which you downloaded this file and run `jupyter notebook`. This should open up a page in your web browser that shows all of the files in the current directory, so that you can open this file. You will need to leave this Terminal window up and running and use a different one for the rest of the instructions.
1. If you didn't install keras previously, install it now
    1. Install the tensorflow machine learning library by typing the following into Terminal:
    `pip3 install --upgrade tensorflow`
    1. Install the keras machine learning library by typing the following into Terminal:
    `pip3 install keras`


## Documentation/Sources
* [Class Notes](https://jennselby.github.io/MachineLearningCourseNotes/#recurrent-neural-networks)
* [https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/) for information on sequence classification with keras
* [https://keras.io/](https://keras.io/) Keras API documentation
* [Keras recurrent tutorial](https://github.com/Vict0rSch/deep_learning/tree/master/keras/recurrent)

# Exercise Option 1: Tune an RNN on the IMDB classification problem

## The IMDB Dataset
The [IMDB dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) consists of movie reviews (x_train) that have been marked as positive or negative (y_train). See the [Word Vectors Tutorial](https://github.com/jennselby/MachineLearningTutorials/blob/master/WordVectors.ipynb) for more details on the IMDB dataset.

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
(imdb_x_train, imdb_y_train), (imdb_x_test, imdb_y_test) = imdb.load_data()

For a standard keras model, every input has to be the same length, so we need to set some length after which we will cutoff the rest of the review. (We will also need to pad the shorter reviews with zeros to make them the same length).

In [3]:
cutoff = 500

In [4]:
imdb_x_train_padded = sequence.pad_sequences(imdb_x_train, maxlen=cutoff)
imdb_x_test_padded = sequence.pad_sequences(imdb_x_test, maxlen=cutoff)

In [5]:
imdb_index_offset = 3 # see https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset

## Classification

In [6]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

Define our model.

Unlike last time, when we used convolutional layers, we're going to use an LSTM, a special type of recurrent network.

Using recurrent networks means that rather than seeing these reviews as one input happening all at one, with the convolutional layers taking into account which words are next to each other, we are going to see them as a sequence of inputs, with one word occurring at each timestep.

In [18]:
imdb_lstm_model = Sequential()
imdb_lstm_model.add(Embedding(input_dim=len(imdb.get_word_index()) + imdb_index_offset,
                              output_dim=100,
                              input_length=cutoff))
# return_sequences tells the LSTM to output the full sequence, for use by the next LSTM layer. The final
# LSTM layer should return only the output sequence, for use in the Dense output layer
imdb_lstm_model.add(LSTM(units=32, return_sequences=True))
imdb_lstm_model.add(LSTM(units=32))
imdb_lstm_model.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
imdb_lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
for layer in imdb_lstm_model.layers:
    print(layer.output_shape)

(None, 500, 100)
(None, 500, 32)
(None, 32)
(None, 1)


Train the model. __This takes 5-15 minutes. You might not want to re-run it unless you are testing out your own changes.__

In [8]:
imdb_lstm_model.fit(imdb_x_train_padded, imdb_y_train, epochs=1, batch_size=64)

Epoch 1/1
12672/25000 [==============>...............] - ETA: 3:11 - loss: 0.5004 - binary_accuracy: 0.7471

KeyboardInterrupt: 

Assess the model. __This takes 2-10 minutes. You might not want to re-run it unless you are testing out your own changes.__

In [ ]:
imdb_lstm_scores = imdb_lstm_model.evaluate(imdb_x_test_padded, imdb_y_test)
print('loss: {} accuracy: {}'.format(*imdb_lstm_scores))

# Exercise Option 1

Experiment with different model configurations from the one above. Try other recurrent layers, different numbers of layers, change some of the defaults. See [Keras Recurrent Layers](https://keras.io/layers/recurrent/)

# Exercise Option 2: Understand the Weight in RNNs

## Exploring Simple Recurrent Layers

Before we dive into something as complicated as LSTMs, Let's take a deeper look at simple recurrent layer weights.

In [ ]:
import numpy
from keras.layers import SimpleRNN
from keras.models import Sequential
from keras.layers import LSTM

The neurons in the recurrent layer pass their output to the next layer, but also back to themselves. The input shape says that we'll be passing in one-dimensional inputs of unspecified length (the None is what makes it unspecified).

In [ ]:
one_unit_SRNN = Sequential()
one_unit_SRNN.add(SimpleRNN(units=1, input_shape=(None, 1), activation='linear', use_bias=False))

In [ ]:
one_unit_SRNN_weights = one_unit_SRNN.get_weights()
one_unit_SRNN_weights

We can set the weights to whatever we want, to test out what happens with different weight values.

In [ ]:
one_unit_SRNN_weights[0][0][0] = 1
one_unit_SRNN_weights[1][0][0] = 1
one_unit_SRNN.set_weights(one_unit_SRNN_weights)
one_unit_SRNN.get_weights()

We can then pass in different input values, to see what the model outputs.

The code below passes in a single sample that has three time steps.

In [ ]:
one_unit_SRNN.predict(numpy.array([ [[3], [3], [7]] ]))

# Exercise 2a
Figure out what the two weights in the one_unit_SRNN model control. Be sure to test your hypothesis thoroughly. Use different weights and different inputs.

## Slightly larger simple recurrent model

In [ ]:
two_unit_SRNN = Sequential()
two_unit_SRNN.add(SimpleRNN(units=2, input_shape=(None, 1), activation='linear', use_bias=False))

In [ ]:
two_unit_SRNN_weights = two_unit_SRNN.get_weights()
two_unit_SRNN_weights

In [ ]:
two_unit_SRNN_weights[0][0][0] = 1
two_unit_SRNN_weights[0][0][1] = 1
two_unit_SRNN_weights[1][0][0] = 0
two_unit_SRNN_weights[1][0][1] = 1
two_unit_SRNN_weights[1][1][0] = 0
two_unit_SRNN_weights[1][1][1] = 1
two_unit_SRNN.set_weights(two_unit_SRNN_weights)
two_unit_SRNN.get_weights()

This passes in a single sample with four time steps.

In [ ]:
two_unit_SRNN.predict(numpy.array([ [[3], [3], [7], [5]] ]))

# Exercise 2b
What do each of the six weights of the two_unit_SRNN control? Again, test out your hypotheses carefully.

## Exploring LSTMs (Optional Extension to Exercise 2)


In [ ]:
one_unit_LSTM = Sequential()
one_unit_LSTM.add(LSTM(units=1, input_shape=(None, 1),
                       activation='linear', recurrent_activation='linear',
                       use_bias=False, unit_forget_bias=False,
                       kernel_initializer='zeros',
                       recurrent_initializer='zeros',
                       return_sequences=True))

In [ ]:
one_unit_LSTM_weights = one_unit_LSTM.get_weights()
one_unit_LSTM_weights

In [ ]:
one_unit_LSTM_weights[0][0][0] = 1
one_unit_LSTM_weights[0][0][1] = 0
one_unit_LSTM_weights[0][0][2] = 1
one_unit_LSTM_weights[0][0][3] = 1
one_unit_LSTM_weights[1][0][0] = 0
one_unit_LSTM_weights[1][0][1] = 0
one_unit_LSTM_weights[1][0][2] = 0
one_unit_LSTM_weights[1][0][3] = 0
one_unit_LSTM.set_weights(one_unit_LSTM_weights)
one_unit_LSTM.get_weights()

In [ ]:
one_unit_LSTM.predict(numpy.array([ [[0], [1], [2], [4]] ]))

# Exercise 2c
Conceptually, the [LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) has several _gates_:

* __Forget gate__: these weights allow some long-term memories to be forgotten.
* __Input gate__: these weights decide what new information will be added to the context cell.
* __Output gate__: these weights decide what pieces of the new information and updated context will be passed on to the output.

It also has a __cell__ that can hold onto information from the current input (as well as things it has remembered from previous inputs), so that it can be used in later outputs.

Identify which weights in the one_unit_LSTM model are connected with the context and which are associated with the three gates. This is considerably more difficult to do by looking at the inputs and outputs, so you could also treat this as a code reading exercise and look through the keras code to find the answer.

_Note_: The output from the predict call is what the linked explanation calls $h_{t}$.

# Exercise Option 3: Set up your own RNN model for the Reuters Classification Problem

Take the model from exercise 1 (imdb_lstm_model) and modify it to classify the [Reuters data](https://keras.io/datasets/#reuters-newswire-topics-classification).

Think about what you are trying to predict in this case, and how you will have to change your model to deal with this.

In [9]:
from keras.datasets import reuters
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Reshape
from keras.utils import to_categorical

In [30]:
(reuters_x_train, reuters_y_train), (reuters_x_test, reuters_y_test) = reuters.load_data()
reuters_x_train_padded = sequence.pad_sequences(reuters_x_train, maxlen=cutoff)
reuters_x_test_padded = sequence.pad_sequences(reuters_x_test, maxlen=cutoff)
reuters_y_train = to_categorical(reuters_y_train)
reuters_y_test = to_categorical(reuters_y_test)

In [11]:
word_index = reuters.get_word_index()
reverse_word_index = {}
for item in word_index:
    reverse_word_index[word_index[item]] = item

space = " "
test = space.join([reverse_word_index.get(i-3,"?") for i in reuters_x_train[0]])
print(test)
print(reuters_y_train[0])

? mcgrath rentcorp said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [33]:
reuters_model = Sequential()
reuters_model.add(Embedding(input_dim=len(reuters.get_word_index()) + imdb_index_offset,
                              output_dim=100,
                              input_length=cutoff))
# return_sequences tells the LSTM to output the full sequence, for use by the next LSTM layer. The final
# LSTM layer should return only the output sequence, for use in the Dense output layer
reuters_model.add(LSTM(units=64, return_sequences=True))
reuters_model.add(LSTM(units=46, return_sequences=True))
reuters_model.add(LSTM(units=46))
reuters_model.add(Dense(units=46, activation='softmax')) # because at the end, we want one yes/no answer
reuters_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# reuters_model.add(Reshape((8982*2,46)))
for layer in reuters_model.layers:
    print(layer.output_shape)

(None, 500, 100)
(None, 500, 64)
(None, 500, 46)
(None, 46)
(None, 46)


In [34]:
reuters_model.fit(reuters_x_train_padded, reuters_y_train, epochs=1, batch_size=64)

Epoch 1/1
8982/8982 [==============================] - 204s 23ms/step - loss: 2.5942 - acc: 0.3480


In [35]:
reuters_scores = reuters_model.evaluate(reuters_x_test_padded, reuters_y_test)
print('loss: {} accuracy: {}'.format(*reuters_scores))

2246/2246 [==============================] - 15s 7ms/step
loss: 2.4209518230927385 accuracy: 0.36197684778237277


In [ ]:
# the model is really bad but at least it works